In [1]:
# -*- coding: utf-8 -*-

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import numpy as np
import time

def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)


/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##################################################
BATCH_SIZE = 100
class_size = 4
learning_rate = 0.001   # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
train_keep_prob = 0.7
max_len = 200           # sequence 단어 수 제한
max_vocab = 40000       # maximum 단어 개수
emb_dim = 1         # 단어 embedding dimension
hidden_dim = 128        # RNN hidden dim
train_acc_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_acc_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
train_loss_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_loss_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
hypothesis_summary = []
##################################################
from ensemble_loader import movie_data
data = movie_data(max_vocab=max_vocab, max_len=max_len)

train_total_batch = int(data.train_size / BATCH_SIZE)
test_total_batch = int(data.test_size / BATCH_SIZE)

from genre_ensemble_model_test import Model
model = Model(max_len=max_len,
              emb_dim=emb_dim,
              hidden_dim=hidden_dim,
              vocab_size=max_vocab,
              class_size=class_size,
              use_clip=True, learning_rate=learning_rate)

sess = initialize_session()
sess.run(tf.global_variables_initializer())

Using TensorFlow backend.


x_train size 4800
y_train size 4800
x_test  size 1200
y_test  size 1200
x_ids_train  size 4800
x_len_train  size 4800
x_ids_test  size 1200
x_len_test  size 1200
1
Tensor("Mean:0", shape=(?, 256), dtype=float32)
Tensor("Reshape:0", shape=(?, 200), dtype=float32)
Tensor("concat:0", shape=(?, 456), dtype=float32)


In [3]:
def test_model():
    test_loss, test_acc = 0, 0

    for _ in range(test_total_batch):
        _, test_x_img, test_ids, test_length, test_y = data.get_test(BATCH_SIZE)

        loss, acc = sess.run([model.loss, model.accuracy],
                              feed_dict={model.x_image: test_x_img, model.x_ids: test_ids, model.x_len: test_length,
                              model.y_label: test_y, model.keep_prob : 1.0, model.is_training:False})
        test_loss += loss
        test_acc += acc

    return test_loss/test_total_batch, test_acc/test_total_batch
    #print(" * test loss: {:.3f} | test acc: {:.3f}\n".format(test_loss / test_cnt, sess.run(model.accuracy, feed_dict={model.x_image: test_x, model.y_label: test_label})))




In [4]:
for epoch in range(100):

    train_avg_loss, train_avg_acc = 0, 0

    for it in range(train_total_batch):
        train_x_img, train_ids, train_length, train_y = data.get_train(BATCH_SIZE)

        loss, acc, _, hypothesis_ = sess.run([model.loss, model.accuracy, model.update, model.genre_prob],
                                              feed_dict={model.x_image: train_x_img, model.x_ids: train_ids, model.x_len: train_length,
                                              model.y_label: train_y, model.keep_prob : train_keep_prob, model.is_training:True})
        train_avg_loss += loss / train_total_batch
        train_avg_acc += acc / train_total_batch

        #if it % 10 == 0 and it > 0:
            #model.save(sess)
        #    print("*Model Saved train_loss: {:.4f}, train_acc: {:.4f}".format(loss/BATCH_SIZE, acc))

    test_avg_loss, test_avg_acc = test_model()

    train_acc_summary.append(train_avg_acc)
    test_acc_summary.append(test_avg_acc)
    train_loss_summary.append(train_avg_loss)
    test_loss_summary.append(test_avg_loss)
    hypothesis_summary.append(hypothesis_)
    #model.save(sess)

    print("epoch {} - train_loss: {:.4f}, train_acc: {:.4f}, test_loss: {:.4f}, test_acc: {:.4f}"
          .format(epoch+1, train_avg_loss, train_avg_acc, test_avg_loss, test_avg_acc))

epoch 1 - train_loss: 1.3124, train_acc: 0.3683, test_loss: 1.2943, test_acc: 0.3717
epoch 2 - train_loss: 1.2584, train_acc: 0.4212, test_loss: 1.2569, test_acc: 0.4300
epoch 3 - train_loss: 1.2013, train_acc: 0.4754, test_loss: 1.2366, test_acc: 0.4458
epoch 4 - train_loss: 1.0960, train_acc: 0.5483, test_loss: 1.1871, test_acc: 0.4767
epoch 5 - train_loss: 0.9404, train_acc: 0.6383, test_loss: 1.1708, test_acc: 0.4775
epoch 6 - train_loss: 0.7793, train_acc: 0.7156, test_loss: 1.1777, test_acc: 0.4858
epoch 7 - train_loss: 0.6272, train_acc: 0.7871, test_loss: 1.2162, test_acc: 0.4925
epoch 8 - train_loss: 0.4964, train_acc: 0.8435, test_loss: 1.2802, test_acc: 0.5000
epoch 9 - train_loss: 0.3879, train_acc: 0.8794, test_loss: 1.3743, test_acc: 0.5008
epoch 10 - train_loss: 0.3085, train_acc: 0.9083, test_loss: 1.4707, test_acc: 0.5000
epoch 11 - train_loss: 0.2439, train_acc: 0.9317, test_loss: 1.5652, test_acc: 0.4833
epoch 12 - train_loss: 0.1894, train_acc: 0.9521, test_loss: 1.

epoch 97 - train_loss: 0.0001, train_acc: 1.0000, test_loss: 4.3536, test_acc: 0.4558
epoch 98 - train_loss: 0.0001, train_acc: 1.0000, test_loss: 4.3579, test_acc: 0.4542
epoch 99 - train_loss: 0.0001, train_acc: 1.0000, test_loss: 4.3671, test_acc: 0.4558
epoch 100 - train_loss: 0.0001, train_acc: 1.0000, test_loss: 4.3759, test_acc: 0.4558


In [ ]:
print('Max Train Accuracy {:.2f}%'.format(np.max(train_acc_summary)))
print('Max Test Accuracy {:.2f}%'.format(np.max(test_acc_summary)))

import matplotlib.pyplot as plt

# Get the figure and the axes
fig, (acc, loss) = plt.subplots(nrows=1,ncols=2, sharey=False, figsize=(15, 5))

# 첫번째 그래프
acc.plot(train_acc_summary, 'r', label='train')
acc.plot(test_acc_summary, 'b', label='test')
acc.legend(loc='upper left', frameon=False)
acc.set(title='Accuracy', xlabel='Epoch', ylabel='Accuracy')

# 두번째 그래프
loss.plot(train_loss_summary, 'r', label='train')
loss.plot(test_loss_summary, 'b', label='test')
loss.legend(loc='upper left', frameon=False)
loss.set(title='Loss', xlabel='Epoch', ylabel='Accuracy')

# Title the figure
fig.suptitle('TEST', fontsize=14, fontweight='bold')